Import libraries

In [1]:
from tqdm import tqdm # To check completion of loops
from geopy.geocoders import Nominatim # To obtain coordinates and address given the name
import folium # For making maps
from folium.plugins import MarkerCluster # To make maps
import pandas as pd # For DataFrame operations

Obtain list of names, coordinates (lat and lon), and addresses under each category (Malls, Restaurants, Offices)

In [2]:
def GetData(name):
    # Define path of file
    MyPath = "commercial.xlsx"
    # Read file
    excel = pd.ExcelFile(MyPath) 
    # Read sheet in file
    df = excel.parse(name, header = None)
    # Preprocess
    if name == "Malls":
        df[0] += " Mall"
    
    AddList = []
    LatList = []
    LonList = []
    
    # Define geolocator 
    geolocator = Nominatim(user_agent = "Akshit Bhalla", timeout = 1000)
    # Obtain required
    for item in tqdm(df[0]):
        location = geolocator.geocode(str(item) + ", Bangalore")
        
        AddList.append(location.address)
        LatList.append(location.latitude) 
        LonList.append(location.longitude)
    
    return(list(df[0].values), LatList, LonList, AddList)

In [3]:
malls, mall_lat, mall_lon, mall_add = GetData("Malls")
restaurants, restaurant_lat, restaurant_lon, restaurant_add = GetData("Restaurants")
# offices, office_lat, office_lon, office_add = GetData("Offices")

100%|██████████| 35/35 [00:18<00:00,  1.88it/s]


Mapping

In [4]:
# Define map
MyMap = folium.Map(
    location = (12.9716, 77.5946), # Coordinates of Bangalore
    zoom_start = 10
)

In [5]:
# Function to make elements to be displayed on the map
def MakeMap(name, cat, lat, lon, add, mc):
    if name == "Mall":
        color = "blue"
    elif name == "Restaurant":
        color = "green"
    else:
        color = "red"
    
    for i in range(35):
#         mc.add_child(
        marker = folium.Marker(
                location = (lat[i], lon[i]),
                popup = (
                    "<b>" + name + ":</b> {}<br>"
                    "<b>Address:</b> {}"
                ).format(cat[i], add[i]),
                icon = folium.Icon(color = color)
            )
#         )
        
    return marker
#     return mc
            

In [6]:
mc1 = MakeMap("Mall", malls, mall_lat, mall_lon, mall_add, MarkerCluster())
mc2 = MakeMap("Restaurant", restaurants, restaurant_lat, restaurant_lon, restaurant_add, MarkerCluster())
# mc3 = MakeMap("Office", offices, office_lat, office_lon, office_add, MarkerCluster())

In [7]:
# Create groups of features
fg1 = folium.FeatureGroup(name = "malls")
fg2 = folium.FeatureGroup(name = "restraunts")
# fg3 = folium.FeatureGroup(name = "offices")
# Add elements to respective groups
mc1.add_to(fg1)
mc2.add_to(fg2)
# mc3.add_to(fg3)
# Add groups to map
MyMap.add_child(fg1)
MyMap.add_child(fg2)
# MyMap.add_child(fg3)
# Display map with layers
MyMap.add_child(folium.LayerControl())

In [8]:
# # Save Map as HTML
# MyMap.save("MyMap.html")

In [9]:
# # Create DataFrame of necessary data
# MyData = pd.DataFrame({
#     "Mall" : malls,
#     "Latitude" : mall_lat,
#     "Longitude" : mall_lon,
#     "Address" : mall_add,
#     "Restaurant" : restaurants,
#     "Latitude" : restaurant_lat,
#     "Longitude" : restaurant_lon,
#     "Address" : restaurant_add,
#     "Office" : offices,
#     "Latitude" : office_lat,
#     "Longitude" : office_lon,
#     "Address" : office_add
# })

In [10]:
# # Export DataFrame
# MyData.to_csv("Commercial_Data.csv") 